In [1]:
from koselleck import *

In [2]:
words=get_valid_words()
# words

In [3]:
pathdf=get_model_paths_df(os.path.join(MODELS_PATH,'bpo'), 'model.txt.gz').sort_values(['period_start','run'])
# pathdf

In [36]:
pathdf_f=pathdf.query('1700<=period_start<1900').groupby('period_start').head(10)
# pathdf_f

In [37]:
def get_neighborhoods(pathdf,topn=10,words=None):
    if words is None: words=get_valid_words()
    word2w2l=dict((w,defaultdict(list)) for w in words)
    iterr=tqdm(pathdf.iterrows(), total=len(pathdf),position=0)
    for i,row in iterr:
        iterr.set_description(f'Loading {row.corpus} ({row.period_start}-{row.period_end}, {row.run})')
        m=load_model(row.path, row.path_vocab)
        for word in set(words)&set(m.key_to_index.keys()):
            top=m.most_similar(word,topn=topn)
            for w,c in top:
                word2w2l[word][w]+=[c]
    return pd.DataFrame([
        {
            'word':word,
            'neighbor':w,
            'num_top':len(l),
            'sum_top':sum(l)
        }
        for word in word2w2l
        for w,l in sorted(word2w2l[word].items(),key=lambda lt: -sum(lt[1]))[:topn]
    ]).sort_values(['word','num_top','sum_top'],ascending=[1,0,0])

In [38]:
# ndf=get_neighborhoods(words,pathdf.iloc[:3])
# ndf

In [39]:
odf=pmap_groups(
    get_neighborhoods,
#     pathdf_f.query('period_start<1730').groupby('period_start'),
    pathdf_f.groupby('period_start'),
    num_proc=3
)

Mapping get_neighborhoods [x3]: 100%|██████████| 20/20 [1:50:27<00:00, 331.40s/it]


In [45]:
odf

,word,neighbor,num_top,sum_top
period_start,,,,
1700,abbe,lettre,3,2.615602
1700,abbe,traite,3,2.570777
1700,abbe,dieu,3,2.536120
1700,abbe,cette,2,1.750145
1700,abbe,chrifto,2,1.726810
...,...,...,...,...
1890,zeal,energy,10,5.701498
1890,zeal,piety,10,5.633949
1890,zeal,earnestness,8,4.543089


In [50]:
odf.to_csv('data/data.neighborhoods.long.csv')

In [46]:
def summarize_neighborhoods(df):
    ol=[]
    for (period,word),pwdf in df.groupby(['period_start','word']):
        pwdf=pwdf.sort_values(['num_top','sum_top'],ascending=False)
        o=', '.join(f'{n} ({t})' for n,t in zip(pwdf.neighbor,pwdf.num_top))
        ol.append({
            'period_start':period,
            'word':word,
            'neighborhood':o
        })
    return pd.DataFrame(ol)

In [42]:
ndfsummary=summarize_neighborhoods(odf)
ndfsummary

,period_start,word,neighborhood
0,1700,abbe,"lettre (3), traite (3), dieu (3), cette (2), chrifto (2), pagg (2), urbis (2), meme (2), guerre (2), depuis (2)"
1,1700,abbey,"angelo (9), treves (4), residing (4), abby (4), sapieha (3), incamp (2), katharine (2), monastery (2), winchester (2), carthage (2)"
2,1700,abbot,"lamberg (6), jean (5), sorbonne (5), mellarede (4), biwhop (4), janfon (4), denis (4), bignon (3), ambrose (3), vatican (3)"
3,1700,abilities,"ability (10), probity (10), integrity (8), pill (8), pills (6), capacities (5), deals (3), drink (3), healths (3), perfous (2)"
4,1700,ability,"abilities (10), probity (8), generosity (3), complaisance (3), pills (3), sagacity (3), knowledg (3), liberality (3), exactness (2), impulse (2)"
...,...,...,...
94747,1890,yields,"yield (10), yielded (10), yielding (10), produces (10), gives (10), loses (8), absorbs (7), brings (7), consumes (6), finds (5)"
94748,1890,yoke,"collar (10), bodice (10), sleeves (9), frill (9), guipure (8), corselet (7), ruching (6), flounce (6), plastron (6), decolletage (4)"
94749,1890,youth,"childhood (10), boyhood (10), youthful (10), manhood (10), girlhood (10), maidenhood (10), boyish (9), infancy (9), womanhood (4), adolescence (3)"
94750,1890,youths,"girls (10), lads (10), boys (10), maidens (10), youth (9), striplings (8), damsels (8), ladies (8), women (7), youngsters (6)"


In [43]:
pd.options.display.max_colwidth=None

In [53]:
ndfsummary[ndfsummary.word=='camera']

,period_start,word,neighborhood
17167,1740,camera,"metaphysics (3), emblematical (3), phyfico (3), obscura (2), delineation (2), parabola (2), contexture (2), physiognomy (2), newtonian (2), smatterer (2)"
21758,1750,camera,"obscura (6), pyrometer (3), sparry (3), multum (3), architeaure (2), coppcr (2), magnetism (2), uvea (2), xther (2), locket (2)"
26466,1760,camera,"mirrors (4), configuration (3), tropes (3), prism (3), antinous (3), smoothness (2), etruscan (2), logarithmic (2), speculums (2), colourings (2)"
31229,1770,camera,"enharmonic (5), adverbs (4), diversifying (3), analysing (3), theorists (2), inversion (2), resemblances (2), menstruum (2), theophrastus (2), microscopes (2)"
36055,1780,camera,"poussin (2), supplements (2), constellation (1), 1s2 (1), de/cription (1), ecclesiastes (1), cuthites (1), colle6tion (1), xxvil (1), derivations (1)"
40902,1790,camera,"mechanifin (2), diationary (2), fraler (1), catlc (1), lectin (1), rcbert (1), walkers (1), personating (1), shiels (1), aeing (1)"
45785,1800,camera,"obscura (9), torpedo (2), nebula (2), vagina (2), bifid (2), chimney-piece (2), rosette (2), photometer (1), sarcoma (1), oculi (1)"
50684,1810,camera,"obscura (7), musica (6), fantasia (5), spina (3), origines (2), concerto (2), gauss (2), breislak (2), angrisani (2), somno (2)"
55603,1820,camera,"obscura (10), lucida (7), secolo (7), venezia (6), italiana (4), saggio (4), storia (3), grammatica (2), museo (2), londra (2)"
60546,1830,camera,"obscura (10), lucida (10), lucilia (10), studio (7), duplex (4), photographic (4), capella (3), spectrum (2), microscopes (2), lapidem (2)"


In [51]:
ndfsummary.to_csv('data/data.neighborhoods.wide.csv')